# Lesson 3: Memory

https://learn.deeplearning.ai/langchain/lesson/3/memory

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory


In [2]:
import os
import warnings

from dotenv import find_dotenv, load_dotenv
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
    ConversationTokenBufferMemory,
)

warnings.filterwarnings("ignore")

In [3]:
load_dotenv(find_dotenv())

try:
    os.environ["OPENAI_API_KEY"]
except Exception:
    raise Exception("No OpenAI key found!")

In [4]:
CHAT_GPT_MODEL = "gpt-3.5-turbo-1106"

### ConverstaionBufferMemory


In [5]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0)

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [6]:
conversation.predict(input="Hi, my name is Joe")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Joe
AI:

> Finished chain.


"Hello Joe! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?"

In [7]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Joe
AI: Hello Joe! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [8]:
conversation.predict(input="Remind me what my name is.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Joe
AI: Hello Joe! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: Remind me what my name is.
AI:

> Finished chain.


'Your name is Joe. Is there anything else you would like to know or need assistance with?'

In [9]:
print(memory.buffer)

Human: Hi, my name is Joe
AI: Hello Joe! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: Remind me what my name is.
AI: Your name is Joe. Is there anything else you would like to know or need assistance with?


In [10]:
memory.load_memory_variables(inputs={})

{'history': "Human: Hi, my name is Joe\nAI: Hello Joe! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: Remind me what my name is.\nAI: Your name is Joe. Is there anything else you would like to know or need assistance with?"}

In [11]:
memory = ConversationBufferMemory()

In [12]:
memory.save_context(inputs={"input": "Hi"}, outputs={"output": "What's up"})

In [13]:
print(memory.buffer)

Human: Hi
AI: What's up


In [14]:
memory.load_memory_variables(inputs={})

{'history': "Human: Hi\nAI: What's up"}

In [15]:
memory.save_context(
    inputs={"input": "Not much, just hanging"}, outputs={"output": "Cool"}
)

In [16]:
memory.load_memory_variables(inputs={})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### ConversationBufferWindowMemory


In [17]:
memory = ConversationBufferWindowMemory(k=1)

In [19]:
memory.save_context(inputs={"input": "Hi"}, outputs={"output": "What's up"})

memory.save_context(
    inputs={"input": "Not much, just hanging"}, outputs={"output": "Cool"}
)

In [20]:
memory.load_memory_variables(inputs={})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [21]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0)

memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

In [22]:
conversation.predict(input="Hi, my name is Bob")

"Hello Bob! It's nice to meet you. My name is AI and I'm here to help you with any questions or tasks you have. How can I assist you today?"

In [23]:
conversation.predict(input="What is 1+1?")

"1+1 equals 2. Is there anything else you'd like to know?"

In [24]:
conversation.predict(input="What's my name again?")

"I'm sorry, I don't have that information. Can you please remind me of your name?"

### ConversationTokenBufferMemory


In [26]:
llm = ChatOpenAI(model=CHAT_GPT_MODEL, temperature=0)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

memory.save_context(inputs={"input": "AI is what?!"}, outputs={"output": "Amazing!"})

memory.save_context(
    inputs={"input": "Backpropagation is what?"}, outputs={"output": "Beautiful!"}
)

memory.save_context(
    inputs={"input": "Chatbots are what?"}, outputs={"output": "Charming!"}
)

In [27]:
memory.load_memory_variables(inputs={})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### ConversationSummaryMemory


In [28]:
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

In [29]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

memory.save_context(inputs={"input": "Hello"}, outputs={"output": "What's up"})

memory.save_context(
    inputs={"input": "Not much, just hanging"}, outputs={"output": "Cool"}
)

memory.save_context(
    inputs={"input": "What is on the schedule today?"},
    outputs={"output": f"{schedule}"},
)

In [30]:
memory.load_memory_variables(inputs={})

{'history': "System: The human and AI exchange greetings and discuss the day's schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer to discuss the latest in AI."}

In [31]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [32]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the day's schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer to discuss the latest in AI.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the product team meeting, a good demo to show would be the latest updates on the LangChain project, including the new language processing algorithms we've implemented. We can also showcase the improved user interface and the enhanced functionality for translating complex sentences. Additionally, we can highlight the performance improvements and the potential impact on user experience. For the lunch meeting with the customer, we can demonstrate the latest advancements in AI, such as natural language understanding and generation, as well as the integration of AI into various applications. We can also discuss the potential use cases and benefits for the customer's specific industry."

In [33]:
memory.load_memory_variables({})

{'history': "System: The human and AI discuss the day's schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer to discuss the latest in AI. The AI suggests demos for the product team meeting, including updates on the LangChain project and improvements in language processing algorithms, user interface, and functionality. For the lunch meeting with the customer, the AI recommends showcasing advancements in AI, natural language understanding and generation, and potential use cases for the customer's industry."}